In [71]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
%matplotlib inline

### The mnist data set
from fuel.datasets import MNIST
def get_data(datatype):
    mnist = MNIST((datatype,))
    state = mnist.open()
    im,_ = mnist.get_data(state=state, request=[i for i in range(mnist.num_examples)] )
    return im.reshape(im.shape[0],28*28)/255
data = get_data('train')
print(data.shape)

(60000, 784)


# PCA

### Derivation

* Find the covariance matrix - an empirical approximation.
* Decompose it into SVD 
    * but with a smaller ???
        * No just pick largest scaling values and corresponding rotations ? rotatio columns = eigenvectors of xTx?
* Pick principle components (/eigenvectors?).
$$
\begin{align}
 \mathbf{Q}_{\mathbf{XY}} = \frac{1}{n-1} \mathbf{M}_{\mathbf{X}}^T \mathbf{M}_{\mathbf{Y}} \\
\end{align}
$$


##### Resources
* https://en.wikipedia.org/wiki/Covariance_matrix
* http://www.cs.otago.ac.nz/cosc453/student_tutorials/principal_components.pdf
* http://mengnote.blogspot.co.nz/2013/05/an-intuitive-explanation-of-pca.html
* [Principle components, minor components and linear networks - Oja](https://users.ics.aalto.fi/oja/Oja92.pdf)
* [Mixtures of Probabilistic Principal Component Analysers](http://www.miketipping.com/papers/met-mppca.pdf)

### Covariance
Why? We want to find how our dimensions vary with each other one. So we need to find how change in one dimension is correlated with the change in others. Another way of saying this is ..

$$
\text{Co-Variance} (X) = X^T \cdot X \\
$$
So let's pick a simple example to play with, a (4,3) matrix of data samples/observations and features/dimensions.
$$
= \begin{bmatrix}
x_{1,1} & x_{2,1} & x_{3,1} \\
x_{1,2} & x_{2,2} & x_{3,2} \\
x_{1,3} & x_{2,3} & x_{3,3} \\
x_{1,4} & x_{2,4} & x_{3,4} \\
\end{bmatrix} 
\begin{bmatrix}
x_{1,1} & x_{1,2} & x_{1,3} & x_{1,4} \\
x_{2,1} & x_{2,2} & x_{2,3} & x_{2,4} \\
x_{3,1} & x_{3,2} & x_{3,3} & x_{3,4} \\
\end{bmatrix} \\
= \begin{bmatrix}
x_{1,1}x_{1,1} + x_{2,1}x_{2,1} + x_{3,1}x_{3,1} & x_{1,1}x_{1,2} + x_{2,1}x_{2,2} + x_{3,1}x_{3,2} & x_{1,1}x_{1,3} + x_{2,1}x_{2,3} + x_{3,1}x_{3,3} & x_{1,1}x_{1,4} + x_{2,1}x_{2,4} + x_{3,1}x_{3,4} \\
x_{1,2}x_{1,1} + x_{2,2}x_{2,1} + x_{3,2}x_{3,1} & x_{1,2}x_{1,2} + x_{2,2}x_{2,2} + x_{3,2}x_{3,2} & x_{1,2}x_{1,3} + x_{2,2}x_{2,3} + x_{3,2}x_{3,3} & x_{1,2}x_{1,4} + x_{2,2}x_{2,4} + x_{3,2}x_{3,4} \\
x_{1,3}x_{1,1} + x_{2,3}x_{2,1} + x_{3,3}x_{3,1} & x_{1,3}x_{1,2} + x_{2,3}x_{2,2} + x_{3,3}x_{3,2} & x_{1,3}x_{1,3} + x_{2,3}x_{2,3} + x_{3,3}x_{3,3} & x_{1,3}x_{1,4} + x_{2,3}x_{2,4} + x_{3,3}x_{3,4} \\
x_{1,4}x_{1,1} + x_{2,4}x_{2,1} + x_{3,4}x_{3,1} & x_{1,4}x_{1,2} + x_{2,4}x_{2,2} + x_{3,4}x_{3,2} & x_{1,4}x_{1,3} + x_{2,4}x_{2,3} + x_{3,4}x_{3,3} & x_{1,4}x_{1,4} + x_{2,4}x_{2,4} + x_{3,4}x_{3,4} \\
\end{bmatrix} \\
$$
Alternatively, we can view this matrix as the product of columns of X. Where $c_i$ indicates the ith column of X.
$$
= \begin{bmatrix}
c_1 \cdot c_1 & c_1 \cdot c_2 & c_1 \cdot c_3 & c_1 \cdot c_4 \\
c_2 \cdot c_1 & c_2 \cdot c_2 & c_2 \cdot c_3 & c_2 \cdot c_4 \\
c_3 \cdot c_1 & c_3 \cdot c_2 & c_3 \cdot c_3 & c_3 \cdot c_4 \\
c_4 \cdot c_1 & c_4 \cdot c_2 & c_4 \cdot c_3 & c_4 \cdot c_4 \\
\end{bmatrix}
$$

So what do entries of the covariance matrix really tell us? 
* Along the **diagonal** we have the sum of the squared values for a given column of X.
    * E.g. ```CoVar(X)[0,0]``` is $x_{1,1}x_{1,1} + x_{2,1}x_{2,1} + x_{3,1}x_{3,1} = c_1 \cdot c_1$
* The rest of the elements (**off-diagonal**) are the dot product of two different columns of X.
    * E.g. ```CoVar(X)[3,0]``` is $x_{1,4}x_{1,1} + x_{2,4}x_{2,1} + x_{3,4}x_{3,1} = c_4\cdot c_1$
    * Which is equivalent to the dot product of column 1 with column 4. 
    
What does the dot product of two vectors mean?
From the definition of variance we have 

Assuming the columns have been whitened (which we will come to later) lets explore the products of some columns.

Now comes the complicated part. $(-) \times (-) = (+), (+) \times (+) = (+), (-) \times (+) = (-)$ We can see that if the two column entries have the same sign, aka are in some binary sense correlated, then they will result in a position value. If not, negative. From this, it is easily seen that the closer the two columns are to being the same, the higher the correlation (remember the columns are whitened, so increasing the value of a column will not necessarily result in higher correlation).

The key to this is the whitening. 

In [56]:
def whiten(x):
    return (x-np.mean(x,0).reshape((1,x.shape[1]))) / np.sqrt(np.var(x,0).reshape((1,x.shape[1]))**2 + 1e-6)
    #return (x-np.mean(x,1).reshape((x.shape[0]),1)) / np.sqrt(np.var(x,1).reshape((x.shape[0],1))**2 + 1e-8)
ims = whiten(data)

In [76]:
my_covar = np.dot(ims.T , ims)*(1/(166*ims.shape[0])) #haha, why 166?
their_covar = np.cov(data.T)
print(np.mean(my_covar[100,0:784]))
print(np.mean(their_covar[100,0:784]))
#hmm...

0.000635718471861
0.000634716982959


1843968

### Singular value decomposition

Is ?

> * The columns of V (right-singular vectors) are eigenvectors of M∗M.
* The columns of U (left-singular vectors) are eigenvectors of MM∗.
* The non-zero elements of Σ (non-zero singular values) are the square roots of the non-zero eigenvalues of M∗M or MM∗.

Show/prove? this

In [65]:
### Using numpy - covariance and SVD
U, S, V = np.linalg.svd(their_covar, full_matrices=True)
print('Principle component variance = ',np.max(S))

Principle component variance =  5.38127233108


In [66]:
### Using sklearn's PCA
pca = PCA(n_components=5,whiten=True)
pca.fit(data)
print('Principle component variance = ',pca.explained_variance_[0])

Principle component variance =  5.38073420384


In [ ]:
#Check how well our model fits to training data vs test data vs noise
print(pca.score(train_ims))
print(pca.score(test_ims))
print(pca.score(np.random.random(test_ims.shape)))
#how can you have positive log-likelihoods?

test_score = pca.score_samples(test_ims)
noise_score = pca.score_samples(np.random.random(test_ims.shape))
print(np.min(test_score))
print(np.max(noise_score))

In [ ]:
# kpca = KernelPCA(kernel='sigmoid')
# kpca.fit(train_ims)

